In [1]:
import pandas as pd
import re
import numpy as np

import yfinance as yf
from yahoofinancials import YahooFinancials
from datetime import date

import matplotlib.pyplot as plt

#Data viz
import plotly.graph_objs as go


In [2]:
ticker = 'AAPL'
fin_data = yf.Ticker(ticker)

In [3]:
data_yf = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = ticker,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "5y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically, Auto Adjust will overwrite the Close price by Adj Close. 
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  1 of 1 completed


In [4]:
data_yf

,Open,High,Low,Close,Volume
Date,,,,,
2017-06-27,34.319020,34.591188,33.990053,34.016087,99047600
2017-06-28,34.195948,34.579347,33.881181,34.513081,88329600
2017-06-29,34.248012,34.347411,33.672910,34.004242,125997600
2017-06-30,34.186482,34.307184,34.027916,34.084717,92096400
2017-07-03,34.288252,34.387651,33.866986,33.961651,57111200
...,...,...,...,...,...
2022-06-21,133.419998,137.059998,133.320007,135.869995,81000500
2022-06-22,134.789993,137.759995,133.910004,135.350006,73409200
2022-06-23,136.820007,138.589996,135.630005,138.270004,72433800


In [5]:
data_yf['MarketCap'] = data_yf['Open'] * data_yf['Volume']

In [6]:
data_yf.head(2)

,Open,High,Low,Close,Volume,MarketCap
Date,,,,,,
2017-06-27,34.319020,34.591188,33.990053,34.016087,99047600,3.399217e+09
2017-06-28,34.195948,34.579347,33.881181,34.513081,88329600,3.020514e+09


In [7]:
#latest financials using the desired ticker and the objects from yahoofinance. 

pnl = fin_data.financials
bs = fin_data.balancesheet
cf = fin_data.cashflow
df_pnl = pd.concat([pnl,bs,cf])

In [8]:
df_pnl.head(2)

,2021-09-25,2020-09-26,2019-09-28,2018-09-29
Research Development,21914000000.0,18752000000.0,16217000000.0,14236000000.0
Effect Of Accounting Charges,None,None,None,None


In [9]:
data_yf.head(2)

,Open,High,Low,Close,Volume,MarketCap
Date,,,,,,
2017-06-27,34.319020,34.591188,33.990053,34.016087,99047600,3.399217e+09
2017-06-28,34.195948,34.579347,33.881181,34.513081,88329600,3.020514e+09


In [10]:
#beautiful object (info) to obtain key info of the company chosen. 
#from here I will take key information but also key figures for further ratio calculaton. 

fin_data.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 154000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience wit

In [11]:
# EPS - calculation from the df and the info. 

data_yf['forwardEPS'] = fin_data.info['forwardEps']
data_yf['trailingEPS'] = fin_data.info['trailingEps']

In [12]:
# PE RATIOS

data_yf['ForwardPE'] = data_yf['Open'] / fin_data.info['forwardEps']
data_yf['TrailingPE'] = data_yf['Open'] / fin_data.info['trailingEps']

In [13]:
# PEG RATIOS 

data_yf['ForwardPEG'] = data_yf['ForwardPE'] / fin_data.info['earningsGrowth']
data_yf['TrailingPEG'] = data_yf['TrailingPE'] / fin_data.info['earningsGrowth']

In [14]:
data_yf.reset_index(inplace = True)

In [15]:
#I dont need these because they are fixed values. 

data_yf.drop(['forwardEPS','trailingEPS'], axis = 1, inplace = True)

In [16]:
data_yf.head(2)

,Date,Open,High,Low,Close,Volume,MarketCap,ForwardPE,TrailingPE,ForwardPEG,TrailingPEG
0,2017-06-27,34.319020,34.591188,33.990053,34.016087,99047600,3.399217e+09,5.239545,5.592149,60.924941,65.024990
1,2017-06-28,34.195948,34.579347,33.881181,34.513081,88329600,3.020514e+09,5.220755,5.572095,60.706459,64.791805


In [17]:
data_yf.to_csv('data_clean.csv', index = False)